# Преобразование датасета из Толоки в датасет для бенчмарка 

In [1]:
import cv2
import pandas as pd
import os


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks


In [3]:
os.listdir('/home/kirillk/Documents/yandex_toloka/ocr_act_2/')

['assignments_from_pool_1183396__24-05-2022.tsv',
 '.~lock.assignments_ocr_act.csv#',
 'assignments_ocr_act.csv',
 'tasks_for_pool_1183396__23-05-2022.tsv']

## Загрузка исходного датасета и первичные преобразования

In [4]:
df = pd.read_csv('/home/kirillk/Documents/yandex_toloka/ocr_act_2/assignments_ocr_act.csv',
                sep='\t')

In [5]:
df.columns = ['filename', 'rectangles', 'doc_type', 'doc_number', 'doc_date']

In [6]:
df['filename'] = df['filename'].apply(lambda x: x.split('/')[-1])

In [7]:
df['scan_is_good'] = True

In [8]:
df.head()

,filename,rectangles,doc_type,doc_number,doc_date,scan_is_good
0,001.jpg,"{""shape"":""rectangle""\,""left"":0.026561342789272...",акт,6017103091,24.11.2021,True
1,002.jpg,"{""shape"":""rectangle""\,""left"":0.037937630530775...",акт,6600174850,12.10.2021,True
2,003.jpg,"{""shape"":""rectangle""\,""left"":0.070513471935820...",акт,6600226583,07.12.2021,True
3,004.jpg,"{""shape"":""rectangle""\,""left"":0.033116105141907...",акт,6600236611,17.12.2021,True
4,005.jpg,"{""shape"":""rectangle""\,""left"":0.039245006144036...",Акт,6600241100,22.12.2021,True


In [9]:
df.shape

(200, 6)

## Подготовка данных в датасете

### Подготовка данных в столбце 'filename'

In [10]:
df['filename'].unique()

array(['001.jpg', '002.jpg', '003.jpg', '004.jpg', '005.jpg', '006.jpg',
       '007.jpg', '008.jpg', '009.jpg', '010.jpg', '011.jpg', '012.jpg',
       '013.jpg', '014.jpg', '015.jpg', '016.jpg', '017.jpg', '018.jpg',
       '019.jpg', '020.jpg', '021.jpg', '022.jpg', '023.jpg', '024.jpg',
       '025.jpg', '026.jpg', '027.jpg', '028.jpg', '029.jpg', '030.jpg',
       '031.jpg', '032.jpg', '033.jpg', '034.jpg', '035.jpg', '036.jpg',
       '037.jpg', '038.jpg', '039.jpg', '040.jpg', '041.jpg', '042.jpg',
       '043.jpg', '044.jpg', '045.jpg', '046.jpg', '047.jpg', '048.jpg',
       '049.jpg', '050.jpg', '051.jpg', '052.jpg', '053.jpg', '054.jpg',
       '055.jpg', '056.jpg', '057.jpg', '058.jpg', '059.jpg', '060.jpg',
       '061.jpg', '062.jpg', '063.jpg', '064.jpg', '065.jpg', '066.jpg',
       '067.jpg', '068.jpg', '069.jpg', '070.jpg', '071.jpg', '072.jpg',
       '073.jpg', '074.jpg', '075.jpg', '076.jpg', '077.jpg', '078.jpg',
       '079.jpg', '080.jpg', '081.jpg', '082.jpg', 

Данные готовы

### Подготовка данных в столбце 'doc_type'

In [11]:
df['doc_type'].unique()

array(['акт', 'Акт'], dtype=object)

In [12]:
df['doc_type'] = df['doc_type'].str.lower()

In [13]:
df['doc_type'].unique()

array(['акт'], dtype=object)

### Подготовка данных в столбце 'doc_number'

In [14]:
df[df['doc_number'].str.len() !=10]

,filename,rectangles,doc_type,doc_number,doc_date,scan_is_good
105,106.jpg,"{""shape"":""rectangle""\,""left"":0.044387646488832...",акт,66000703223,01.07.2021,True
146,147.jpg,"{""shape"":""rectangle""\,""left"":0.040854104431064...",акт,660018923,06.10.2021,True
178,179.jpg,"{""shape"":""rectangle""\,""left"":0.046268894950537...",акт,66002121,19.11.2021,True


Исправим ошибки разметки.

In [15]:
df['doc_number'][105] = '6600070323'
df['doc_number'][105]

/tmp/ipykernel_67950/3288290856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['doc_number'][105] = '6600070323'


'6600070323'

In [16]:
df['doc_number'][146] = '6600168923'
df['doc_number'][146]

/tmp/ipykernel_67950/3781156153.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['doc_number'][146] = '6600168923'


'6600168923'

In [17]:
df['doc_number'][178] = '6600212138'
df['doc_number'][178]

/tmp/ipykernel_67950/3648073837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['doc_number'][178] = '6600212138'


'6600212138'

In [18]:
df[df['doc_number'].str.len() !=10]

,filename,rectangles,doc_type,doc_number,doc_date,scan_is_good


In [19]:
df[df['doc_number'].str.isnumeric() == False]

,filename,rectangles,doc_type,doc_number,doc_date,scan_is_good
13,014.jpg,"{""shape"":""rectangle""\,""left"":0.034684013379357...",акт,????412637,12.08.2021,True


В этом документе не читатется номер

### Парсинг столбца rectangles

In [20]:
rec = df['rectangles'][0]

In [21]:
rec = eval(rec.replace('\\', ''))

In [22]:
rec

({'shape': 'rectangle',
  'left': 0.026561342789272125,
  'top': 0.014445423244476877,
  'width': 0.023020131736874765,
  'height': 0.008173871708341935,
  'label': 'doc_type'},
 {'shape': 'rectangle',
  'left': 0.06649778345400183,
  'top': 0.013454650916192998,
  'width': 0.07132752939683164,
  'height': 0.01015541636490969,
  'label': 'doc_number'},
 {'shape': 'rectangle',
  'left': 0.15665996609009467,
  'top': 0.01333080437515752,
  'width': 0.06539810152521239,
  'height': 0.009536183659732257,
  'label': 'doc_date'})

In [23]:
example = pd.DataFrame(rec).set_index('label').drop('shape', axis=1).T.unstack().to_frame().T

In [24]:
example

label  doc_type                              doc_number                      \
           left       top    width    height       left       top     width   
0      0.026561  0.014445  0.02302  0.008174   0.066498  0.013455  0.071328   

label           doc_date                                
         height     left       top     width    height  
0      0.010155  0.15666  0.013331  0.065398  0.009536

In [25]:
example.columns

MultiIndex([(  'doc_type',   'left'),
            (  'doc_type',    'top'),
            (  'doc_type',  'width'),
            (  'doc_type', 'height'),
            ('doc_number',   'left'),
            ('doc_number',    'top'),
            ('doc_number',  'width'),
            ('doc_number', 'height'),
            (  'doc_date',   'left'),
            (  'doc_date',    'top'),
            (  'doc_date',  'width'),
            (  'doc_date', 'height')],
           names=['label', None])

In [26]:
example.columns = ['_'.join(col) for col in example.columns]
example.columns

Index(['doc_type_left', 'doc_type_top', 'doc_type_width', 'doc_type_height',
       'doc_number_left', 'doc_number_top', 'doc_number_width',
       'doc_number_height', 'doc_date_left', 'doc_date_top', 'doc_date_width',
       'doc_date_height'],
      dtype='object')

In [27]:
'_'.join(example.columns[0].split('_')[:2])

'doc_type'

In [28]:
new_columns_list = []

In [29]:
for col in example.columns:
    new_columns_list.append('_'.join(col.split('_')[:2]) + '_box_' + col.split('_')[-1])
#     print(col)

In [30]:
new_columns_list

['doc_type_box_left',
 'doc_type_box_top',
 'doc_type_box_width',
 'doc_type_box_height',
 'doc_number_box_left',
 'doc_number_box_top',
 'doc_number_box_width',
 'doc_number_box_height',
 'doc_date_box_left',
 'doc_date_box_top',
 'doc_date_box_width',
 'doc_date_box_height']

In [31]:
example.columns = new_columns_list

In [32]:
example.columns = example.columns.str.replace('-', '_')

In [33]:
example.columns

Index(['doc_type_box_left', 'doc_type_box_top', 'doc_type_box_width',
       'doc_type_box_height', 'doc_number_box_left', 'doc_number_box_top',
       'doc_number_box_width', 'doc_number_box_height', 'doc_date_box_left',
       'doc_date_box_top', 'doc_date_box_width', 'doc_date_box_height'],
      dtype='object')

In [34]:
example

,doc_type_box_left,doc_type_box_top,doc_type_box_width,doc_type_box_height,doc_number_box_left,doc_number_box_top,doc_number_box_width,doc_number_box_height,doc_date_box_left,doc_date_box_top,doc_date_box_width,doc_date_box_height
0,0.026561,0.014445,0.02302,0.008174,0.066498,0.013455,0.071328,0.010155,0.15666,0.013331,0.065398,0.009536


In [35]:
rectangles_list = []

In [36]:
for rectangles in df['rectangles']:
    rectangles = eval(rectangles.replace('\\', ''))
    rectangles = pd.DataFrame(rectangles).set_index('label').drop('shape', axis=1).T.unstack().to_frame().T
    
    rectangles.columns = ['_'.join(col) for col in rectangles.columns]
    
    new_columns_list = []
    for col in rectangles.columns:
        new_columns_list.append('_'.join(col.split('_')[:2]) + '_box_' + col.split('_')[-1])
    
    rectangles.columns = new_columns_list
    
    rectangles.columns = rectangles.columns.str.replace('-', '_')
    rectangles.columns = rectangles.columns.str.replace('name', 'type')
    rectangles_list.append(rectangles)

In [37]:
boxes_table = pd.concat(rectangles_list, ignore_index=True)

In [38]:
boxes_table.head(10)

,doc_type_box_left,doc_type_box_top,doc_type_box_width,doc_type_box_height,doc_number_box_left,doc_number_box_top,doc_number_box_width,doc_number_box_height,doc_date_box_left,doc_date_box_top,doc_date_box_width,doc_date_box_height
0,0.026561,0.014445,0.023020,0.008174,0.066498,0.013455,0.071328,0.010155,0.156660,0.013331,0.065398,0.009536
1,0.037938,0.035595,0.023487,0.007534,0.079714,0.034500,0.072964,0.008492,0.172122,0.034500,0.065841,0.008766
2,0.070513,0.037068,0.023613,0.008848,0.111159,0.036792,0.073161,0.008709,0.202126,0.036239,0.065419,0.009401
3,0.033116,0.045036,0.023148,0.008422,0.073842,0.045036,0.071531,0.008422,0.164170,0.044789,0.065962,0.008669
4,0.039245,0.047049,0.023848,0.007750,0.080277,0.047049,0.072069,0.007625,0.170582,0.046799,0.066808,0.008125
5,0.067706,0.042772,0.022930,0.007247,0.108695,0.041613,0.072238,0.008406,0.199602,0.041613,0.064122,0.009131
6,0.037542,0.015661,0.024504,0.009043,0.079905,0.014623,0.075173,0.009784,0.174806,0.015068,0.066867,0.009636
7,0.062008,0.017558,0.023240,0.008174,0.102635,0.016815,0.074368,0.009536,0.194734,0.016567,0.067654,0.009288
8,0.061624,0.034692,0.024034,0.010413,0.101997,0.034968,0.073431,0.009451,0.194266,0.035242,0.065165,0.009725
9,0.052598,0.036906,0.024313,0.008183,0.093661,0.036267,0.074200,0.008566,0.186771,0.036012,0.065195,0.009589


In [39]:
df = df.join(boxes_table)

In [40]:
df.head()

,filename,rectangles,doc_type,doc_number,doc_date,scan_is_good,doc_type_box_left,doc_type_box_top,doc_type_box_width,doc_type_box_height,doc_number_box_left,doc_number_box_top,doc_number_box_width,doc_number_box_height,doc_date_box_left,doc_date_box_top,doc_date_box_width,doc_date_box_height
0,001.jpg,"{""shape"":""rectangle""\,""left"":0.026561342789272...",акт,6017103091,24.11.2021,True,0.026561,0.014445,0.023020,0.008174,0.066498,0.013455,0.071328,0.010155,0.156660,0.013331,0.065398,0.009536
1,002.jpg,"{""shape"":""rectangle""\,""left"":0.037937630530775...",акт,6600174850,12.10.2021,True,0.037938,0.035595,0.023487,0.007534,0.079714,0.034500,0.072964,0.008492,0.172122,0.034500,0.065841,0.008766
2,003.jpg,"{""shape"":""rectangle""\,""left"":0.070513471935820...",акт,6600226583,07.12.2021,True,0.070513,0.037068,0.023613,0.008848,0.111159,0.036792,0.073161,0.008709,0.202126,0.036239,0.065419,0.009401
3,004.jpg,"{""shape"":""rectangle""\,""left"":0.033116105141907...",акт,6600236611,17.12.2021,True,0.033116,0.045036,0.023148,0.008422,0.073842,0.045036,0.071531,0.008422,0.164170,0.044789,0.065962,0.008669
4,005.jpg,"{""shape"":""rectangle""\,""left"":0.039245006144036...",акт,6600241100,22.12.2021,True,0.039245,0.047049,0.023848,0.007750,0.080277,0.047049,0.072069,0.007625,0.170582,0.046799,0.066808,0.008125


### Проверка join

In [41]:
for idx, num in enumerate(df['doc_type_box_left']):
    if str(num) not in df['rectangles'][idx]:
        print(idx, 'NOT ok')

### Готовый датасет

In [42]:
df = df.drop('rectangles', axis=1)

In [43]:
df.head(10)

,filename,doc_type,doc_number,doc_date,scan_is_good,doc_type_box_left,doc_type_box_top,doc_type_box_width,doc_type_box_height,doc_number_box_left,doc_number_box_top,doc_number_box_width,doc_number_box_height,doc_date_box_left,doc_date_box_top,doc_date_box_width,doc_date_box_height
0,001.jpg,акт,6017103091,24.11.2021,True,0.026561,0.014445,0.023020,0.008174,0.066498,0.013455,0.071328,0.010155,0.156660,0.013331,0.065398,0.009536
1,002.jpg,акт,6600174850,12.10.2021,True,0.037938,0.035595,0.023487,0.007534,0.079714,0.034500,0.072964,0.008492,0.172122,0.034500,0.065841,0.008766
2,003.jpg,акт,6600226583,07.12.2021,True,0.070513,0.037068,0.023613,0.008848,0.111159,0.036792,0.073161,0.008709,0.202126,0.036239,0.065419,0.009401
3,004.jpg,акт,6600236611,17.12.2021,True,0.033116,0.045036,0.023148,0.008422,0.073842,0.045036,0.071531,0.008422,0.164170,0.044789,0.065962,0.008669
4,005.jpg,акт,6600241100,22.12.2021,True,0.039245,0.047049,0.023848,0.007750,0.080277,0.047049,0.072069,0.007625,0.170582,0.046799,0.066808,0.008125
5,006.jpg,акт,6600146317,14.09.2021,True,0.067706,0.042772,0.022930,0.007247,0.108695,0.041613,0.072238,0.008406,0.199602,0.041613,0.064122,0.009131
6,007.jpg,акт,6600210316,17.11.2021,True,0.037542,0.015661,0.024504,0.009043,0.079905,0.014623,0.075173,0.009784,0.174806,0.015068,0.066867,0.009636
7,008.jpg,акт,6600217949,25.11.2021,True,0.062008,0.017558,0.023240,0.008174,0.102635,0.016815,0.074368,0.009536,0.194734,0.016567,0.067654,0.009288
8,009.jpg,акт,6600223046,02.12.2021,True,0.061624,0.034692,0.024034,0.010413,0.101997,0.034968,0.073431,0.009451,0.194266,0.035242,0.065165,0.009725
9,010.jpg,акт,6600226214,04.12.2021,True,0.052598,0.036906,0.024313,0.008183,0.093661,0.036267,0.074200,0.008566,0.186771,0.036012,0.065195,0.009589


## Проверка на изображениях

In [44]:
mapping_prefix_value = {
    'doc_number_box_': 'doc_number',
    'doc_date_box_': None,
    'doc_type_box_': None,
}

In [55]:
for index, row in df.iterrows():
    if index > 1:
        image = cv2.imread('/home/kirillk/datasets/OCR/ocr_act_1/' + row['filename'])
        image = cv2.resize(image, (0, 0), fx=0.3, fy=0.3)
        height, width, _ = image.shape

        for prefix, value in mapping_prefix_value.items():
            x1 = int(row[prefix + 'left'] * width)
            y1 = int(row[prefix + 'top'] * height)
            x2 = int((row[prefix + 'left'] + row[prefix + 'width']) * width)
            y2 = int((row[prefix + 'top'] + row[prefix + 'height']) * height)

            image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 4)
            image = cv2.putText(image, 
                                str(row.get(value, default='')), 
                                (x1, y2 - 70),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                #0,
                                1,
                                (0, 0, 255),
                                2)

            cv2.imshow('Document', image)

            key = cv2.waitKey(0) & 0xFF

            if key == ord('q'):
                break
    
    cv2.destroyAllWindows()
    if index == 5:
        break

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   filename               200 non-null    object 
 1   doc_type               200 non-null    object 
 2   doc_number             200 non-null    object 
 3   doc_date               200 non-null    object 
 4   scan_is_good           200 non-null    bool   
 5   doc_type_box_left      200 non-null    float64
 6   doc_type_box_top       200 non-null    float64
 7   doc_type_box_width     200 non-null    float64
 8   doc_type_box_height    200 non-null    float64
 9   doc_number_box_left    200 non-null    float64
 10  doc_number_box_top     200 non-null    float64
 11  doc_number_box_width   200 non-null    float64
 12  doc_number_box_height  200 non-null    float64
 13  doc_date_box_left      200 non-null    float64
 14  doc_date_box_top       200 non-null    float64
 15  doc_da

## Сохранение датасета

In [57]:
df.to_csv('/home/kirillk/datasets/OCR/ocr_act_1/test-cases-act.csv')

In [58]:
test = pd.read_csv('/home/kirillk/datasets/OCR/ocr_act_1/test-cases-act.csv')

In [59]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             200 non-null    int64  
 1   filename               200 non-null    object 
 2   doc_type               200 non-null    object 
 3   doc_number             200 non-null    object 
 4   doc_date               200 non-null    object 
 5   scan_is_good           200 non-null    bool   
 6   doc_type_box_left      200 non-null    float64
 7   doc_type_box_top       200 non-null    float64
 8   doc_type_box_width     200 non-null    float64
 9   doc_type_box_height    200 non-null    float64
 10  doc_number_box_left    200 non-null    float64
 11  doc_number_box_top     200 non-null    float64
 12  doc_number_box_width   200 non-null    float64
 13  doc_number_box_height  200 non-null    float64
 14  doc_date_box_left      200 non-null    float64
 15  doc_da